In [2]:
import requests
import json
import pandas as pd 
from datetime import datetime
from importlib import reload 
import gzip 
import connection


connection = reload(connection) 


<connection object at 0x7323fd1e9f80; dsn: 'user=admin password=xxx dbname=dev host=coinbase-cluster.ctpfzvbnyqdb.us-east-1.redshift.amazonaws.com port=5439', closed: 0>
<connection object at 0x7323f3ef2fc0; dsn: 'user=postgres password=xxx dbname=exchange_rate host=172.17.0.3 port=5432', closed: 0>
<connection object at 0x7323f3ef3100; dsn: 'user=admin password=xxx dbname=dev host=coinbase-cluster.ctpfzvbnyqdb.us-east-1.redshift.amazonaws.com port=5439', closed: 0>
<connection object at 0x7323f3ef3240; dsn: 'user=postgres password=xxx dbname=exchange_rate host=172.17.0.3 port=5432', closed: 0>


In [2]:
# # create connection object postgres db 
# local_con =connection.local_pgcon
# local_cur = connection.local_pgcur

# print(local_con) 
# print("\n")
# print(local_cur) 


<connection object at 0x784c3cac2ac0; dsn: 'user=postgres password=xxx dbname=coinbase10 host=172.17.0.2 port=5432', closed: 0>


<built-in method cursor of psycopg2.extensions.connection object at 0x784c3cac2ac0>


CREATE CONNECTIONS TO AWS S3, AWS REDSHIFT, LOCAL POSTGRES DB

In [2]:
# create s3 object
s3 = connection.s3_bucket

# create Redshift connection and cursor
cur_redshift = connection.cur_redshift
conn_redshift = connection.conn_redshift


#create local postgres connection and cursor
cur_pg = connection.cur_pg
conn_pg = connection.conn_pg



In [60]:
# open exchange data, url is found in config.py


response = requests.get(connection.api_url)         

In [168]:
# Assign the API response to the variable api_data
api_data = response.text
# api_data

'{\n  "disclaimer": "Usage subject to terms: https://openexchangerates.org/terms",\n  "license": "https://openexchangerates.org/license",\n  "timestamp": 1716120007,\n  "base": "USD",\n  "rates": {\n    "AED": 3.673,\n    "AFN": 71.479051,\n    "ALL": 92.148131,\n    "AMD": 386.030742,\n    "ANG": 1.792758,\n    "AOA": 849,\n    "ARS": 881.603415,\n    "AUD": 1.492315,\n    "AWG": 1.8025,\n    "AZN": 1.7,\n    "BAM": 1.795053,\n    "BBD": 2,\n    "BDT": 116.488448,\n    "BGN": 1.79898,\n    "BHD": 0.375109,\n    "BIF": 2853.833157,\n    "BMD": 1,\n    "BND": 1.340645,\n    "BOB": 6.87348,\n    "BRL": 5.1046,\n    "BSD": 1,\n    "BTC": 0.000014849738,\n    "BTN": 82.888175,\n    "BWP": 13.478959,\n    "BYN": 3.254922,\n    "BZD": 2.00514,\n    "CAD": 1.36175,\n    "CDF": 2775.346609,\n    "CHF": 0.911919,\n    "CLF": 0.032417,\n    "CLP": 894.49773,\n    "CNH": 7.2317,\n    "CNY": 7.2232,\n    "COP": 3807.1681,\n    "CRC": 509.113839,\n    "CUC": 1,\n    "CUP": 25.75,\n    "CVE": 101.20

MAKE TRANSFORMATIONS TO THE API DATA: DATE COLUMN CREATION, RATE_DATE VARIABLE TO APPEND TO CSV FILE NAME

In [169]:
if response.status_code == 200:  
    api_data = f''' {api_data} '''
    api_data
    api_data = json.loads(api_data)
    # api_data
    df = pd.DataFrame(api_data)

    # extract base and timestamp column data
    base = api_data['base']
    timestamp = api_data['timestamp']

    # Get date and time from timestamp 
    date = datetime.fromtimestamp(timestamp)
    # format date to YYYYMMDD to be used as a column in the dataframe
    year = date.strftime('%Y%m%d')
    # format time to HMS
    time = date.strftime('%H%M%S')
    # this will be part of the csv filename to ensure uniqueness of filename 
    rate_date = year + '_'+ time

    df = pd.DataFrame(api_data['rates'].items(), columns=['Currency', 'Rate'])

    # Add base, timestamp, and date columns
    df['Base'] = base
    df['Timestamp'] = timestamp 
    df['Date'] = year
    print("API Data Successfully Extracted")
else:
    ("Failed to retrieve data. Status code:", response.status_code)



In [1]:
# File name to be used for saving csv, exchange_rate + date and time from the timestamp of the dataframe
csv_file_path = f'./datasets/exchange_rate_{rate_date}.csv' 
print(csv_file_path)
df.to_csv(csv_file_path, index=False)

NameError: name 'rate_date' is not defined

In [152]:
# creating a filename variable for compressed file to be uploaded to S3
file_name = csv_file_path + '.gz'
print(file_name)

exchange_rate_test_20151211-235001.csv.gz


In [154]:
# Compress CSV to zip format and assign the compressed filename to compressed_csv

# location for local storage
compressed_csv_local = file_name

# formatting the location to be stored as datasets/exchange_rate_date_time.csv.gz for s3 bucket
compressed_csv_s3 = file_name[2:]

print(compressed_csv_s3)

DATA UPLOAD TO AWS S3 BUCKET 

In [155]:
print("*****S3 DATA UPLOAD")
with open(compressed_csv_local, 'rb') as file:
    # Upload the file object to S3 bucket
    s3.upload_fileobj(file, 'emil-coinbase-bucket', compressed_csv_s3)

In [156]:
# Check if the file has been successfully uploaded
if s3.head_object(Bucket='emil-coinbase-bucket', Key=compressed_csv_s3):
    print("CSV file uploaded to S3 bucket successfully")
else:
    print("Upload not successful")

CSV file uploaded to S3 bucket successfully


CREATE LOCAL DATABASE TABLE, AND INSERT DATA INTO DATABASE

In [5]:
table_name = 'exchange_rates'
try:
    
    rates_table = f'''CREATE TABLE IF NOT EXISTS {table_name} (
                        currency VARCHAR(3),
                        rate NUMERIC(16, 8),
                        base VARCHAR(3),
                        timestamp VARCHAR(16),
                        date VARCHAR(8)
                    )'''
    connection.cur_pg.execute(rates_table)
    connection.conn_pg.commit()
except conn_pg.Error as e:
    connection.conn_pg.rollback()
    print("Error occurred:", e)




In [26]:
# INSERT DATA TO LOCAL DB
try:
    # size of each batch
    batch_size = 200

    # Insert DataFrame into database in batches
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i + batch_size]
        values = [tuple(row) for row in batch_df.values]

        placeholders = ','.join(['%s'] * len(df.columns))
        try:
            insert_query = f'''INSERT INTO {table_name} (currency, rate, base, timestamp, date) 
                   VALUES ({placeholders})'''
                   

            connection.cur_pg.executemany(insert_query, values)
            connection.conn_pg.commit()

            print(insert_query)
        except Exception as e:
            # Rollback the transaction
            connection.conn_pg.rollback()


except Exception as e:
    print("An error occurred during batch data insertion:", e)
    connection.conn_pg.rollback()  # Rollback the transaction in case of an error


REDSHIFT TABLE CREATION AND FILE COPY FROM S3 BUCKET

In [6]:
    # create table REDSHIFT

table_name = 'exchange_rates'
try:
    
 rates_table = f'''CREATE TABLE IF NOT EXISTS {table_name} (
                        currency VARCHAR(3),
                        rate NUMERIC(16, 8),
                        base VARCHAR(3),
                        timestamp VARCHAR(16),
                        date VARCHAR(8)
                    )'''
    
  
 connection.cur_redshift.execute(rates_table)
 connection.conn_redshift.commit()
 print("Table created successfully.")

except Exception as e:
 connection.conn_redshift.rollback()
 print(f"Error creating table: {e}")

In [125]:
# upload data to Redshift form s3 bucket
   

aws_access_key_id = connection.aws_access_key_id
aws_secret_access_key = connection.aws_secret_access_key

try:
    insert_from_s3 = f"""COPY {table_name}
    FROM 's3://emil-coinbase-bucket/{compressed_csv_s3}'
    CREDENTIALS 'aws_access_key_id={aws_access_key_id};aws_secret_access_key={aws_secret_access_key}'
    IGNOREHEADER 1
    FORMAT AS csv
    gzip
    DELIMITER ','; """
    
    connection.cur_redshift.execute(insert_from_s3)
    connection.conn_redshift.commit()
    print(f"Copy to Redshift: {table_name} was successful")

except Exception as e:
    connection.conn_redshift.rollback()
    print(f"An error occurred: {str(e)}")

finally:
    # Close the Redshift connection
    connection.conn_redshift.close()





INSERT INTO currency_rates (ID, Name, min_size) VALUES (%s,%s,%s)
